## Transfer Learning

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/BoostcampAITech/lecture-note-python-basics-for-ai/blob/main/codes/pytorch/07_torch-study/transfer_learning/transfer_learning.ipynb)

In [1]:
!pip install torchsummary

  Using cached torchsummary-1.5.1-py3-none-any.whl (2.8 kB)


## Saving and Loading Model 

https://pytorch.org/tutorials/beginner/saving_loading_models.html

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import os

In [3]:
class TheModelClass(nn.Module):
    def __init__(self):
        super(TheModelClass, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, stride=2, padding=0),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))

        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=3, stride=2, padding=0),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.layer3 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=0),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))

        self.drop_out = nn.Dropout()
        self.fc1 = nn.Linear(3 * 3 * 64, 1000)
        self.fc2 = nn.Linear(1000, 1) # 마지막에 linear를 놓아 0, 1로 예측하도록 (개나 고양이)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)

        out = out.view(out.size(0), -1)
        out = self.drop_out(out)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

In [5]:
# Initialize model
model = TheModelClass()

# Initialize optimizer
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9)

In [6]:
# Print model's state_dict
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

Model's state_dict:
layer1.0.weight 	 torch.Size([16, 3, 3, 3])
layer1.0.bias 	 torch.Size([16])
layer1.1.weight 	 torch.Size([16])
layer1.1.bias 	 torch.Size([16])
layer1.1.running_mean 	 torch.Size([16])
layer1.1.running_var 	 torch.Size([16])
layer1.1.num_batches_tracked 	 torch.Size([])
layer2.0.weight 	 torch.Size([32, 16, 3, 3])
layer2.0.bias 	 torch.Size([32])
layer2.1.weight 	 torch.Size([32])
layer2.1.bias 	 torch.Size([32])
layer2.1.running_mean 	 torch.Size([32])
layer2.1.running_var 	 torch.Size([32])
layer2.1.num_batches_tracked 	 torch.Size([])
layer3.0.weight 	 torch.Size([64, 32, 3, 3])
layer3.0.bias 	 torch.Size([64])
layer3.1.weight 	 torch.Size([64])
layer3.1.bias 	 torch.Size([64])
layer3.1.running_mean 	 torch.Size([64])
layer3.1.running_var 	 torch.Size([64])
layer3.1.num_batches_tracked 	 torch.Size([])
fc1.weight 	 torch.Size([1000, 576])
fc1.bias 	 torch.Size([1000])
fc2.weight 	 torch.Size([1, 1000])
fc2.bias 	 torch.Size([1])


ordered dict tyoe으로 나온다!
- ordered dict tyoe : 데이터의 순서가 보장되는 dict 타입

In [7]:
type(model.state_dict())

collections.OrderedDict

In [8]:
from torchsummary import summary
summary(model, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 16, 111, 111]             448
       BatchNorm2d-2         [-1, 16, 111, 111]              32
              ReLU-3         [-1, 16, 111, 111]               0
         MaxPool2d-4           [-1, 16, 55, 55]               0
            Conv2d-5           [-1, 32, 27, 27]           4,640
       BatchNorm2d-6           [-1, 32, 27, 27]              64
              ReLU-7           [-1, 32, 27, 27]               0
         MaxPool2d-8           [-1, 32, 13, 13]               0
            Conv2d-9             [-1, 64, 6, 6]          18,496
      BatchNorm2d-10             [-1, 64, 6, 6]             128
             ReLU-11             [-1, 64, 6, 6]               0
        MaxPool2d-12             [-1, 64, 3, 3]               0
          Dropout-13                  [-1, 576]               0
           Linear-14                 [-

In [9]:
MODEL_PATH ="saved"
if not os.path.exists(MODEL_PATH):
    os.makedirs(MODEL_PATH) # saved 폴더 만들어주기
torch.save(model.state_dict(), 
           os.path.join(MODEL_PATH, "model.pt")) # model.pt로 state_dict 저장

In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

new_model = TheModelClass() # 동일한 모델을 만들어
new_model.load_state_dict(torch.load(os.path.join(
    MODEL_PATH, "model.pt"))) # load_state_dict

<All keys matched successfully>

In [11]:
torch.save(model, os.path.join(MODEL_PATH, "model_pickle.pt")) # 모델 자체를 save
model = torch.load(os.path.join(MODEL_PATH, "model_pickle.pt"))
model.eval()

TheModelClass(
  (layer1): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (drop_out): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=576, out_features=1000, bias=True)
  (fc2): Linear(in_features=1000, out_features=1, b

In [14]:
import zipfile

filename = "kagglecatsanddogs_5340.zip"
with zipfile.ZipFile(filename, 'r') as zip_ref:
    zip_ref.extractall()

import shutil
shutil.move('PetImages', 'data')

import os
from os import walk

mypath = "data"

f_path = []
for (dirpath, dirnames, filenames) in walk(mypath):
    f_path.extend([os.path.join(dirpath, filename) for filename in filenames])

from PIL import Image

for f_name in f_path:
    try: # 이미지가 이상한 경우, 제거
        Image.open(f_name)
    except Exception as e:
        print(e)
        os.remove(f_name)


cannot identify image file 'data/PetImages/Cat/Thumbs.db'
cannot identify image file 'data/PetImages/Cat/666.jpg'


/Users/lee/opt/anaconda3/envs/DL/lib/python3.8/site-packages/PIL/TiffImagePlugin.py:781: UserWarning: Possibly corrupt EXIF data.  Expecting to read 32 bytes but only got 0. Skipping tag 270
  warnings.warn(
/Users/lee/opt/anaconda3/envs/DL/lib/python3.8/site-packages/PIL/TiffImagePlugin.py:781: UserWarning: Possibly corrupt EXIF data.  Expecting to read 5 bytes but only got 0. Skipping tag 271
  warnings.warn(
/Users/lee/opt/anaconda3/envs/DL/lib/python3.8/site-packages/PIL/TiffImagePlugin.py:781: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 272
  warnings.warn(
/Users/lee/opt/anaconda3/envs/DL/lib/python3.8/site-packages/PIL/TiffImagePlugin.py:781: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 282
  warnings.warn(
/Users/lee/opt/anaconda3/envs/DL/lib/python3.8/site-packages/PIL/TiffImagePlugin.py:781: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0

cannot identify image file 'data/PetImages/Dog/Thumbs.db'
cannot identify image file 'data/PetImages/Dog/11702.jpg'


In [16]:
from torchvision import datasets
import torchvision.transforms as transforms
import torch

dataset = datasets.ImageFolder(root="data",
                           transform=transforms.Compose([
                               transforms.Scale(244),       
                               transforms.CenterCrop(244),  
                               transforms.ToTensor(),       
                               transforms.Normalize((0.5, 0.5, 0.5),
                                                    (0.5, 0.5, 0.5)), 
                           ]))
dataloader = torch.utils.data.DataLoader(dataset,
                                         batch_size=8,
                                         shuffle=True,
                                         num_workers=8)

In [15]:
import warnings
warnings.filterwarnings("ignore")

In [17]:
def binary_acc(y_pred, y_test): # binary로 accuracy 구할 때 쓰는 코드
    y_pred_tag = torch.round(torch.sigmoid(y_pred))

    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(acc * 100)
    
    return acc

In [18]:
EPOCHS = 100
BATCH_SIZE = 64
LEARNING_RATE = 0.1

In [19]:
model.to(device)

criterion = nn.BCEWithLogitsLoss() # Binary classification cross entropy - WithLogitsLoss를 붙여 맨 끝에 sigmoid function을 달아줌!
## 앞선 모델에서는 0과 1로만 분류해줬었지, sigmoid function이 따로 존재하지 않았음 -> 
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [22]:
for e in range(1, EPOCHS+1):
    epoch_loss = 0
    epoch_acc = 0
    for X_batch, y_batch in dataloader: # 데이터로더를 가져와 불러온 모델에 배치 파일을 넣어
        # X_batch, y_batch = X_batch.to(device), y_batch.to(device).type(torch.cuda.FloatTensor)
        
        optimizer.zero_grad()        
        y_pred = model(X_batch)
               
        loss = criterion(y_pred, y_batch.unsqueeze(1))
        acc = binary_acc(y_pred, y_batch.unsqueeze(1))
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    # epoch 단위로 dict 저장
    torch.save({
        'epoch': e,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': epoch_loss,
        }, f"saved/checkpoint_model_{e}_{epoch_loss/len(dataloader)}_{epoch_acc/len(dataloader)}.pt")
        

    print(f'Epoch {e+0:03}: | Loss: {epoch_loss/len(dataloader):.5f} | Acc: {epoch_acc/len(dataloader):.3f}')

RuntimeError: result type Float can't be cast to the desired output type Long

## Pretrained Model Loading

#### 추가
- vgg16는 ImageNet으로 학습되어 1000개를 classify하게 된다. 이를 가지고 개와 고양이를 분류하는 모델을 만들고자 한다
- `이전 데이터는 freeze하고, 마지막 softmax layer만 학습`하게 된다
- 이러한 pretraiend 모델을 이용해서 다른 이미지를 학습 시킬 때 좀 더 성능을 높이는 방법이 일단 처음에는 `feature extractor의 weight값을 학습하지 않고 별도의 classification layer(앞에서 말씀드린 2개의 결과 값으로 새롭게 만들어진 fully connected layer)부터 먼저 일정 수준 학습`을 시킨 다음에,  다음에는 `feature extractor를 포함한 전체를 학습하는 방법` https://www.inflearn.com/questions/104722/yolov3-%ED%95%99%EC%8A%B5-freeze-%EA%B4%80%EB%A0%A8
  - 일단 feature extraction의 부분을 멈추고, 그에 기반한 결과값을 나의 layer가 학습하도록 함 -> 일정 수준 학습이 된 이후, feature extraction도 포함하여 학습을 전체적으로 시킨다

In [23]:
import torch
from torchvision import models

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
vgg = models.vgg16(pretrained=True).to(device)
# pretrained=True, 이전에 학습된 모델 가져옴
print(vgg)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /Users/lee/.cache/torch/hub/checkpoints/vgg16-397923af.pth
4.5%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

11.0%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

16.7%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_ra

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [24]:
from torchsummary import summary
summary(vgg, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
              ReLU-2         [-1, 64, 224, 224]               0
            Conv2d-3         [-1, 64, 224, 224]          36,928
              ReLU-4         [-1, 64, 224, 224]               0
         MaxPool2d-5         [-1, 64, 112, 112]               0
            Conv2d-6        [-1, 128, 112, 112]          73,856
              ReLU-7        [-1, 128, 112, 112]               0
            Conv2d-8        [-1, 128, 112, 112]         147,584
              ReLU-9        [-1, 128, 112, 112]               0
        MaxPool2d-10          [-1, 128, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]         295,168
             ReLU-12          [-1, 256, 56, 56]               0
           Conv2d-13          [-1, 256, 56, 56]         590,080
             ReLU-14          [-1, 256,

In [25]:
for name, layer in vgg.named_modules(): # named_modules : 각각의 네임과 레이어 확인 가능
    print(name, layer) # 각 feature별로 접근하는 법도 있다

 VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=

In [27]:
vgg.fc = torch.nn.Linear(1000, 1)
# Linear를 해서 내가 분류하고 싶은 데이터의 개수만큼, 현재는 binary 문제이므로 1
  # multiple loss로 할 경우, 1이 아닌 label 개수에 따라 증가할 것
# vgg.cuda()

In [28]:
vgg = models.vgg16(pretrained=True).to(device)
print(vgg)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

마지막이 (6): Linear(in_features=4096, out_features=1000, bias=True)이여서 맨 뒤에 torch.nn.Linear(1000, 1)을 추가했지만,
이 대신 기존 모델에 있는 마지막 레이어를 하나로 예측하게끔 변환 시킬 수 있다!
- 그런데 보통은 마지막에 layer을 하나 더 추가해주는게 일반적이다! 기존 모델을 바꾸지 않는 것이 좋을 것

In [29]:
vgg.classifier._modules['6'] = torch.nn.Linear(4096, 1)
# vgg.cuda()

## Pretrained Model

In [30]:
from torchvision import datasets
import torchvision.transforms as transforms
import torch

dataset = datasets.ImageFolder(root="data/",
                           transform=transforms.Compose([
                               transforms.Scale(244),       
                               transforms.CenterCrop(244),  
                               transforms.ToTensor(),       
                               transforms.Normalize((0.5, 0.5, 0.5),
                                                    (0.5, 0.5, 0.5)), 
                           ]))
dataloader = torch.utils.data.DataLoader(dataset,
                                         batch_size=2,
                                         shuffle=True,
                                         num_workers=8)

In [31]:
from torch import nn
from torchvision import models

class MyNewNet(nn.Module):   
    def __init__(self):
        super(MyNewNet, self).__init__()
        self.vgg19 = models.vgg19(pretrained=True)
        self.linear_layers = nn.Linear(1000, 1)

    # Defining the forward pass    
    def forward(self, x):
        x = self.vgg19(x)        
        return self.linear_layers(x)

In [32]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

my_model = MyNewNet()
my_model = my_model.to(device)

Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /Users/lee/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth
4.3%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

10.9%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

18.2%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_ra

In [35]:
print(my_model)

MyNewNet(
  (vgg19): VGG(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inplace=True)
      (16):

In [36]:
for param in my_model.parameters():
    param.requires_grad = False

모든 레이어의 파라미터를 frozen 시킨 후,

In [37]:
for param in my_model.linear_layers.parameters():
    param.requires_grad = True

마지막의 파라미터만 requires_grad 가능케 한다! (frozen X, 학습할 것임)

In [38]:
my_model.eval()

MyNewNet(
  (vgg19): VGG(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inplace=True)
      (16):

In [39]:
x = next(iter(dataloader)) # 데이터를 불러옴

In [52]:
x # 데이터로더의 배치가 2였기 먜문에 각 x안 데이터를 2개 불러온다 x[0] : 2개, x[1] : 2개

[tensor([[[[-0.6392, -0.6157, -0.6000,  ..., -0.4980, -0.4980, -0.4824],
           [-0.6392, -0.6235, -0.6157,  ..., -0.4902, -0.4902, -0.4745],
           [-0.6392, -0.6392, -0.6314,  ..., -0.4824, -0.4824, -0.4667],
           ...,
           [ 0.5451,  0.5216,  0.5059,  ..., -0.2235, -0.2078, -0.2157],
           [ 0.5608,  0.5373,  0.5059,  ..., -0.2235, -0.2078, -0.2157],
           [ 0.5529,  0.5529,  0.5451,  ..., -0.2314, -0.2235, -0.2314]],
 
          [[-0.6314, -0.6078, -0.5922,  ..., -0.5059, -0.5059, -0.5059],
           [-0.6314, -0.6157, -0.6078,  ..., -0.4980, -0.4980, -0.4980],
           [-0.6314, -0.6314, -0.6235,  ..., -0.4902, -0.4902, -0.4902],
           ...,
           [ 0.5922,  0.5686,  0.5451,  ..., -0.2627, -0.2471, -0.2549],
           [ 0.6078,  0.5843,  0.5529,  ..., -0.2627, -0.2471, -0.2549],
           [ 0.5922,  0.5922,  0.5765,  ..., -0.2706, -0.2627, -0.2706]],
 
          [[-0.6784, -0.6549, -0.6392,  ..., -0.5451, -0.5451, -0.5451],
           [-

In [40]:
x[0].shape

torch.Size([2, 3, 244, 244])

In [41]:
my_model(x[0].to(device)) # linear 값이기 떄문에 그냥 실수값이 나온다
# 이후에, BCEWithLogitLoss로 변형을 시켜주는 작업을 거친다

tensor([[0.4881],
        [0.2364]], grad_fn=<AddmmBackward0>)

In [47]:
x[1] # 0과 0이므로, 실제 정답의 경우, 두개의 사진 모두 고양이이다!

tensor([0, 0])

In [48]:
EPOCHS = 100
BATCH_SIZE = 64
LEARNING_RATE = 0.001

In [49]:
dataloader = torch.utils.data.DataLoader(dataset,
                                         batch_size=BATCH_SIZE,
                                         shuffle=True,
                                         num_workers=8)

현재의 파라미터값 확인

In [50]:
next(my_model.vgg19.features._modules['0'].parameters())[0]

tensor([[[-0.0535, -0.0493, -0.0679],
         [ 0.0153,  0.0451,  0.0021],
         [ 0.0362,  0.0200,  0.0199]],

        [[ 0.0170,  0.0554, -0.0062],
         [ 0.1416,  0.2271,  0.1376],
         [ 0.1200,  0.2003,  0.0921]],

        [[-0.0449,  0.0127, -0.0145],
         [ 0.0597,  0.1395,  0.0541],
         [-0.0010,  0.0583, -0.0297]]])

In [51]:
it = my_model.linear_layers.parameters()
next(it)[0][:10]

tensor([-0.0191,  0.0037, -0.0276,  0.0049,  0.0143, -0.0300,  0.0218, -0.0148,
         0.0030,  0.0281], grad_fn=<SliceBackward0>)

my_model의 파라미터 경우, frozen했기 떄문에 파라미터가 변하지 않을 것이고, 마지막 linear의 경우 frozen하지 않아서 학습이 됐을 것

In [53]:
def binary_acc(y_pred, y_test):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))

    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(acc * 100)
    
    return acc

새로운 모델을 받아오는 경우는 아까 파라미터 frozen해주는 코드를 그대로 사용해야 한다

In [54]:
my_model = MyNewNet() # 새로운 모델 만들기
my_model = my_model.to(device) # device에 올려주기

# frozen
for param in my_model.parameters():
    param.requires_grad = False
for param in my_model.linear_layers.parameters():
    param.requires_grad = True

# 평가지표 & optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(my_model.parameters(), lr=LEARNING_RATE)


for e in range(1, EPOCHS+1):
    epoch_loss = 0
    epoch_acc = 0
    for X_batch, y_batch in dataloader:
        # X_batch, y_batch = X_batch.to(device), y_batch.to(device).type(torch.cuda.FloatTensor)
        
        optimizer.zero_grad()        
        y_pred = my_model(X_batch)
               
        loss = criterion(y_pred, y_batch.unsqueeze(1))
        acc = binary_acc(y_pred, y_batch.unsqueeze(1))
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        

    print(f'Epoch {e+0:03}: | Loss: {epoch_loss/len(dataloader):.5f} | Acc: {epoch_acc/len(dataloader):.3f}')

RuntimeError: result type Float can't be cast to the desired output type Long

In [55]:
next(my_model.vgg19.features._modules['0'].parameters())[0] # frozen시켰기 떄문에 아까와 동일한 값을 가지는 것을 볼 수 있다

tensor([[[-0.0535, -0.0493, -0.0679],
         [ 0.0153,  0.0451,  0.0021],
         [ 0.0362,  0.0200,  0.0199]],

        [[ 0.0170,  0.0554, -0.0062],
         [ 0.1416,  0.2271,  0.1376],
         [ 0.1200,  0.2003,  0.0921]],

        [[-0.0449,  0.0127, -0.0145],
         [ 0.0597,  0.1395,  0.0541],
         [-0.0010,  0.0583, -0.0297]]])

In [56]:
it = my_model.linear_layers.parameters() # 
next(it)[0][:10]

tensor([ 0.0280, -0.0222, -0.0294, -0.0126,  0.0218, -0.0039,  0.0098,  0.0198,
        -0.0006,  0.0209], grad_fn=<SliceBackward0>)